# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import os
import pickle

import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC

from copy import deepcopy

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
print(os.getcwd())

/home/workspace


In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns
#y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

In [9]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [ ]:
Y.info()

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # normalized text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Extract all the urls from the provided text
    detected_urls = re.findall(url_regex, text)
    # Replace url with a url placeholder string
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    # Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    # List of clean tokens
    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
#using MultiOutputClassifier to build an ML pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,min_samples_leaf=1,max_features='auto',n_jobs=-1))),  
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [15]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((19662,), (6554,), (19662, 36), (6554, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [47]:
y_pred = pipeline.predict(X_test)

In [48]:
y_pred = pd.DataFrame(y_pred)
type(y_pred)

pandas.core.frame.DataFrame

In [49]:
type(Y_test)

pandas.core.frame.DataFrame

In [39]:
type(category_names)

pandas.core.indexes.base.Index

In [52]:
print(classification_report(Y_test, y_pred, target_names= category_names))

ValueError: Unknown label type: (       related  request  offer  aid_related  medical_help  medical_products  \
4371         1        0      0            0             0                 0   
21519        1        0      0            1             1                 1   
24520        0        0      0            0             0                 0   
16336        0        0      0            0             0                 0   
439          1        1      0            1             0                 0   
4212         1        0      0            0             0                 0   
5103         1        0      0            1             0                 0   
9699         1        0      0            0             0                 0   
19153        0        0      0            0             0                 0   
2649         1        1      0            1             0                 0   
14816        1        1      0            1             1                 0   
21543        0     

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ff2c17d0b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10],
             'clf__estimator__min_samples_split': [2, 4],}

cv = GridSearchCV(pipeline, parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 2)
np.random.seed(42)
y_pred2 = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total=  11.1s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total=  11.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total=  11.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, total=  11.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, total=  11.1s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf_

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 20.4min finished


In [24]:
prediction2 = y_pred2.predict(X_test)

print(classification_report(Y_test, prediction2 , target_names = category_names))

ValueError: Unknown label type: (       related  request  offer  aid_related  medical_help  medical_products  \
7917         1        0      0            0             0                 0   
25322        1        0      0            0             0                 0   
22191        1        0      0            1             0                 0   
18442        0        0      0            0             0                 0   
1336         0        0      0            0             0                 0   
24449        1        0      1            1             1                 0   
7976         0        0      0            0             0                 0   
17210        1        0      0            0             0                 0   
14652        1        0      0            1             0                 0   
20339        0        0      0            0             0                 0   
9317         0        0      0            0             0                 0   
25097        1     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
       ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

pipeline2.fit(X_train, Y_train)

In [ ]:
y_pred3 = pipeline2.predict(X_test)

print(classification_report(Y_test, y_pred3, target_names = category_names))

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
#pickle.dump(y_pred2, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.